In [ ]:
import numpy as np
import pandas as pd

def print_vector(vector):
    for i in vector:
        print(f"{i.squeeze():05.3f}") 
    
def convert_to_homogeneous(x):
    """
    Convert 2xn or 3xn vector of n points in euclidean coordinates
    to a 3xn or 4xn vector homogeneous by adding a row of ones
    """
    x = np.array(x)
    ndim, npts = x.shape
    if ndim != 2 and ndim != 3:
        print(
            "Error: wrong number of dimension of the input vector.\
              A number of dimensions (rows) of 2 or 3 is required."
        )
        return None
    x1 = np.concatenate((x, np.ones((1, npts), "float32")), axis=0)
    return x1


def convert_from_homogeneous(x):
    """
    Convert 3xn or 4xn vector of n points in homogeneous coordinates
    to a 2xn or 3xn vector in euclidean coordinates, by dividing by the
    homogeneous part of the vector (last row) and removing one dimension
    """
    x = np.array(x)
    ndim, npts = x.shape
    if ndim != 3 and ndim != 4:
        print(
            "Error: wrong number of dimension of the input vector.\
              A number of dimensions (rows) of 2 or 3 is required."
        )
        return None
    x1 = x[: ndim - 1, :] / x[ndim - 1, :]
    return x1
    
def apply_transformation(T: np.ndarray, x: np.ndarray) -> np.ndarray:

    return T @ x



def get_coordinates_from_df(df: pd.DataFrame, to_homogeneous: bool = False,) -> np.ndarray:
    xyz =  df[["X", "Y", "Z"]].to_numpy()
    if to_homogeneous:
        xyz = np.concatenate((xyz, np.ones((xyz.shape[0],1))), axis=1)
    return xyz
    
T_loc2wgs = np.array(
    [[0.706579327583,	-0.70687371492,	-0.00012600114,	416614.833],
    [0.706873714924,	0.706579267979,	0.000202054813,	5090932.706],
    [-0.00005382637,	-0.00023195939,	0.999462246895,	1767.547],
    [0.,	0.,	0.,	1.0]])

T_wgs2loc = np.linalg.inv(T_loc2wgs)

In [ ]:
# Test
p = np.array(
    [146.765,100.001,88.992,1]
).reshape(4,-1)

out = T_loc2wgs@p
print_vector(out)
print_vector(T_wgs2loc @ out)

In [ ]:
# UTM 2 LOC
p = np.array(
    [
        [416623.40, 5091366.28, 1902.50, 1],
        [416651.77, 5091110.06, 1859.25, 1]
        ]
).T

out = T_wgs2loc@p
print_vector(out[:,0])
print_vector(out[:,1])


In [ ]:
# LOC 2 UTM
p = np.array([
    [312.8893994092904, 300.5131404068843, 135.1131123531500, 1],
[151.60327996073528, 99.2543602448616, 91.64333053419362, 1]]
).T

out = T_loc2wgs@p
print_vector(out[:,0])
print_vector(out[:,1])



In [ ]:
fname = "target_for_belpy_local.txt"
targets = pd.read_csv(fname)

xyz = get_coordinates_from_df(targets, to_homogeneous=True)

utm = T_loc2wgs @ xyz.T
utm = utm.T
targets["E"] = utm[:,0]
targets["N"] = utm[:,1]
targets["h"] = utm[:,2]

targets.loc[0, ["E"]] = 416514.0983
targets.loc[0, ["N"]] = 5091103.5718
targets.loc[0, ["h"]] = 1839.1975

targets_utm = targets[["#Label", "E", "N", "h"]]
targets_utm.to_csv( "target_utm.csv")


In [ ]:
F10_loc = np.array([-1.8212855433302195, 219.2598752180853, 74.55843482880829, 1]).reshape(4,1)

F10_utm = T_loc2wgs @ F10_loc
print_vector(F10_utm)